<a href="https://colab.research.google.com/github/sharsulkar/H1B_LCA_outcome_prediction/blob/main/notebooks/01_sh_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [143]:
%time
#data=pd.read_excel('https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2020_Q2.xlsx')
data=pd.read_excel('https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/PERM_Disclosure_Data_FY2020_Q2.xlsx')

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


In [144]:
data.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,REFILE,ORIG_FILE_DATE,PREVIOUS_SWA_CASE_NUMBER_STATE,SCHD_A_SHEEPHERDER,EMPLOYER_NAME,EMPLOYER_ADDRESS_1,EMPLOYER_ADDRESS_2,EMPLOYER_CITY,EMPLOYER_STATE_PROVINCE,EMPLOYER_COUNTRY,EMPLOYER_POSTAL_CODE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,EMPLOYER_NUM_EMPLOYEES,EMPLOYER_YEAR_COMMENCED_BUSINESS,NAICS_CODE,FW_OWNERSHIP_INTEREST,EMP_CONTACT_NAME,EMP_CONTACT_ADDRESS_1,EMP_CONTACT_ADDRESS_2,EMP_CONTACT_CITY,EMP_CONTACT_STATE_PROVINCE,EMP_CONTACT_COUNTRY,EMP_CONTACT_POSTAL_CODE,EMP_CONTACT_PHONE,EMP_CONTACT_EMAIL,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_FIRM_NAME,AGENT_ATTORNEY_PHONE,AGENT_ATTORNEY_PHONE_EXT,AGENT_ATTORNEY_ADDRESS_1,AGENT_ATTORNEY_ADDRESS_2,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE_PROVINCE,AGENT_ATTORNEY_COUNTRY,AGENT_ATTORNEY_POSTAL_CODE,...,EMPLOYEE_REFERRAL_PROG_TO_DATE,CAMPUS_PLACEMENT_FROM_DATE,CAMPUS_PLACEMENT_TO_DATE,LOCAL_ETHNIC_PAPER_FROM_DATE,LOCAL_ETHNIC_PAPER_TO_DATE,RADIO_TV_AD_FROM_DATE,RADIO_TV_AD_TO_DATE,EMP_RECEIVED_PAYMENT,PAYMENT_DETAILS,BARGAINING_REP_NOTIFIED,POSTED_NOTICE_AT_WORKSITE,LAYOFF_IN_PAST_SIX_MONTHS,US_WORKERS_CONSIDERED,COUNTRY_OF_CITIZENSHIP,FOREIGN_WORKER_BIRTH_COUNTRY,CLASS_OF_ADMISSION,FOREIGN_WORKER_EDUCATION,FOREIGN_WORKER_EDUCATION_OTHER,FOREIGN_WORKER_INFO_MAJOR,FOREIGN_WORKER_YRS_ED_COMP,FOREIGN_WORKER_INST_OF_ED,FOREIGN_WORKER_ED_INST_ADD_1,FOREIGN_WORKER_ED_INST_ADD_2,FOREIGN_WORKER_ED_INST_CITY,FOREIGN_WORKER_ED_INST_STATE_P,FOREIGN_WORKER_ED_INST_COUNTRY,FOREIGN_WORKER_ED_INST_POST_CD,FOREIGN_WORKER_TRAINING_COMP,FOREIGN_WORKER_REQ_EXPERIENCE,FOREIGN_WORKER_ALT_ED_EXP,FOREIGN_WORKER_ALT_OCC_EXP,FOREIGN_WORKER_EXP_WITH_EMPL,FOREIGN_WORKER_EMPL_PAY_FOR_ED,FOREIGN_WORKER_CURR_EMPLOYED,EMPLOYER_COMPLETED_APPLICATION,PREPARER_NAME,PREPARER_TITLE,PREPARER_EMAIL,EMP_INFO_DECL_NAME,EMP_INFO_DECL_TITLE
0,A-10119-96737,Certified,2010-03-29,2019-10-18 11:34:37,Y,2000-09-15,MARYLAND,N,FIL PARONG,13528 JAMIESON PL,NaN,GERMANTOWN,MARYLAND,UNITED STATES OF AMERICA,20874,301-515-8079,NaN,0.0,2009.0,531110,N,FIL PARONG,13528 JAMIESON PL,NaN,GERMANTOWN,MARYLAND,UNITED STATES OF AMERICA,20874,(301) 515-8079,NaN,WINSTON W TSAI,"WINSTON TSAI, ESQ.",(301) 320- 1888,NaN,6202 WINSTON DRIVE,NaN,BETHESDA,MARYLAND,UNITED STATES OF AMERICA,20817,...,NaT,NaT,NaT,NaN,NaT,NaT,NaT,N,NaN,NaN,Y,N,NaN,PHILIPPINES,PHILIPPINES,B-2,High School,NaN,HIGH SCHOOL,1991.0,SAN JACINTO NATIONAL HIGH SCHOOL,10 BARANGAY GUIBEL ST,NaN,SAN JACINTO,BARANGAY,PHILIPPINES,NaN,NaN,Y,NaN,NaN,N,N,N,N,WINSTON W TSAI,ATTORNEY AT LAW,WINSTONTSAI@AOL.COM,FIL PARONG,HEAD OF HOUSEHOLD
1,A-11158-83757,Certified,2019-08-15,2019-11-05 09:27:01,N,NaT,NaN,N,"OMRON AUTOMOTIVE ELECTRONICS, INC.",3709 OHIO AVENUE,NaN,ST. CHARLES,ILLINOIS,UNITED STATES OF AMERICA,60174,630-443-6800,208,718.0,1992.0,334419,N,Jennifer Dobbs,3709 OHIO AVENUE,NaN,ST. CHARLES,ILLINOIS,UNITED STATES OF AMERICA,60174,630-443-7610 208,jennifer.dobbs@omron.com,Fazila Vaid,"Masuda, Funai, Eifert Mitchell, Ltd.",847.734.8811,NaN,200 N. Martingale,Suite 800,Schaumburg,ILLINOIS,UNITED STATES OF AMERICA,60173-5485,...,NaT,2019-05-09,2019-05-20,2019-05-23 00:00:00,2019-05-23,NaT,NaT,N,NaN,NaN,Y,N,NaN,INDIA,INDIA,H-1B,Bachelor's,NaN,ELECTRONIC ENGINEERING,2011.0,GUJARAT UNIVERSITY,NAVRANGPURA,NaN,AHMEDABAD,GUJARAT,INDIA,380009,NaN,Y,NaN,Y,N,N,Y,N,FAZILA VAID,ATTORNEY,fvaid@masudafunai.com,Jennifer Dobbs,Human Resources Administrator
2,A-11187-90575,Certified,2011-08-31,2019-11-01 19:31:47,N,NaT,NaN,N,ITECH US INC.,20 KIMBALL AVENUE,SUITE 303 NORTH,SOUTH BURLINGTON,VERMONT,UNITED STATES OF AMERICA,5403,802-383-1500,123,370.0,2001.0,541511,N,Kate A Keough,20 Kimball Avenue,Suite 303 North,South Burlington,VERMONT,UNITED STATES OF AMERICA,5403,802-383-1500*111 110,kate@itechus.com,Andrea M Wisner,"iTech US, Inc.",727-365-2209,109.0,20 Kimball Avenue,Suite 303 North,South Burlington,VERMONT,UNITED STATES OF AMERICA,5403,...,2011-07-07,NaT,NaT,NaN,NaT,NaT,NaT,N,NaN,NaN,Y,N,NaN,INDIA,INDIA,H-1B,Master's,NaN,CIVIL ENGINEERING,2008.0,THE UN

In [145]:
#create a copy of input data to avoid reimporting it in case the working copy gets corrupted. 
data_copy=data.copy()

In [146]:
#Dictionary to store notes on preprocessing steps
dict_notes={}

In [147]:
#store feature statictics in a dataframe
df_data_statistics=pd.DataFrame(data=None,index=data.columns,columns=['Dtype','percent_missing','cardinality'])

In [148]:
#write reusable function to display
#record count per class
print('Count per class:',data.groupby(['CASE_STATUS']).size())
#df shape - records and features
print('Feature shape:', data.shape)

Count per class: CASE_STATUS
Certified            30260
Certified-Expired      917
Denied                1917
Withdrawn             1735
dtype: int64
Feature shape: (34829, 154)


In [149]:
dict_notes['output_class']='Unbalanced classes'

In [150]:
#dataframe statistics
for column in data.columns:
  #identify numeric, non-numeric and date columns
  df_data_statistics.Dtype.loc[column]=data[column].dtype
  #% missing data for each column
  df_data_statistics.percent_missing.loc[column]=(data.shape[0]-data[column].count())*100/data.shape[0]
  #Cardinality of each column
  df_data_statistics.cardinality.loc[column]=(data.shape[0]-len(data[column].unique()))*100/data.shape[0]

In [151]:
#features with missing values >50%
df_data_statistics[df_data_statistics.percent_missing>=40.0].index

Index(['ORIG_FILE_DATE', 'PREVIOUS_SWA_CASE_NUMBER_STATE',
       'EMPLOYER_ADDRESS_2', 'EMPLOYER_PHONE_EXT', 'EMP_CONTACT_ADDRESS_2',
       'AGENT_ATTORNEY_PHONE_EXT', 'PW_SOURCE_NAME_OTHER', 'WAGE_OFFER_TO',
       'WORKSITE_ADDRESS_2', 'JOB_EDUCATION_MIN_OTHER',
       'REQUIRED_TRAINING_MONTHS', 'REQUIRED_FIELD_OF_TRAINING',
       'REQUIRED_EXPERIENCE_MONTHS', 'ACCEPT_ALT_MAJOR_FLD_OF_STUDY',
       'ACCEPT_ALT_COMBO_EDUCATION', 'ACCEPT_ALT_COMBO_ED_OTHER',
       'ACCEPT_ALT_COMBO_EDUCATION_YRS', 'FOREIGN_WORKER_LIVE_IN_DOM_CNT',
       'COMPETITIVE_PROCESS', 'BASIC_RECRUITMENT_PROCESS',
       'TEACHER_SELECT_DATE', 'TEACHER_PUB_JOURNAL_NAME',
       'ADD_RECRUIT_INFORMATION', 'JOB_FAIR_FROM_DATE', 'JOB_FAIR_TO_DATE',
       'ON_CAMPUS_RECRUITING_FROM_DATE', 'ON_CAMPUS_RECRUITING_TO_DATE',
       'EMPLOYER_WEBSITE_FROM_DATE', 'EMPLOYER_WEBSITE_TO_DATE',
       'PRO_ORG_AD_FROM_DATE', 'PRO_ORG_ADVERTISEMENT_TO_DATE',
       'PVT_EMPLOYMENT_FIRM_FROM_DATE', 'PVT_EMPLOYMENT_FIRM_T

In [152]:
dict_notes['drop_missing']=list(df_data_statistics[df_data_statistics.percent_missing>=40.0].index)

In [153]:
#features with high cardinality
df_data_statistics[df_data_statistics.cardinality<80.0].index

Index(['CASE_NUMBER', 'DECISION_DATE', 'EMPLOYER_NAME', 'EMPLOYER_ADDRESS_1',
       'EMPLOYER_PHONE', 'EMP_CONTACT_NAME', 'EMP_CONTACT_ADDRESS_1',
       'EMP_CONTACT_PHONE', 'EMP_CONTACT_EMAIL', 'PW_TRACK_NUMBER',
       'WAGE_OFFER_FROM', 'WORKSITE_ADDRESS_1', 'JOB_TITLE',
       'MAJOR_FIELD_OF_STUDY', 'ACCEPT_ALT_JOB_TITLE', 'SPECIFIC_SKILLS',
       'FOREIGN_WORKER_INST_OF_ED', 'FOREIGN_WORKER_ED_INST_ADD_1',
       'EMP_INFO_DECL_NAME'],
      dtype='object')

In [154]:
dict_notes['drop_high_carinal']=list(df_data_statistics[df_data_statistics.cardinality<80.0].index)

In [155]:
dict_notes['feature_eng']='Preparer same as Attorney-Y/N?,Worksite address same as employer address?-Y/N'


In [156]:
dict_notes

{'drop_high_carinal': ['CASE_NUMBER',
  'DECISION_DATE',
  'EMPLOYER_NAME',
  'EMPLOYER_ADDRESS_1',
  'EMPLOYER_PHONE',
  'EMP_CONTACT_NAME',
  'EMP_CONTACT_ADDRESS_1',
  'EMP_CONTACT_PHONE',
  'EMP_CONTACT_EMAIL',
  'PW_TRACK_NUMBER',
  'WAGE_OFFER_FROM',
  'WORKSITE_ADDRESS_1',
  'JOB_TITLE',
  'MAJOR_FIELD_OF_STUDY',
  'ACCEPT_ALT_JOB_TITLE',
  'SPECIFIC_SKILLS',
  'FOREIGN_WORKER_INST_OF_ED',
  'FOREIGN_WORKER_ED_INST_ADD_1',
  'EMP_INFO_DECL_NAME'],
 'drop_missing': ['ORIG_FILE_DATE',
  'PREVIOUS_SWA_CASE_NUMBER_STATE',
  'EMPLOYER_ADDRESS_2',
  'EMPLOYER_PHONE_EXT',
  'EMP_CONTACT_ADDRESS_2',
  'AGENT_ATTORNEY_PHONE_EXT',
  'PW_SOURCE_NAME_OTHER',
  'WAGE_OFFER_TO',
  'WORKSITE_ADDRESS_2',
  'JOB_EDUCATION_MIN_OTHER',
  'REQUIRED_TRAINING_MONTHS',
  'REQUIRED_FIELD_OF_TRAINING',
  'REQUIRED_EXPERIENCE_MONTHS',
  'ACCEPT_ALT_MAJOR_FLD_OF_STUDY',
  'ACCEPT_ALT_COMBO_EDUCATION',
  'ACCEPT_ALT_COMBO_ED_OTHER',
  'ACCEPT_ALT_COMBO_EDUCATION_YRS',
  'FOREIGN_WORKER_LIVE_IN_DOM_CNT',
  